In [1]:
import numpy as np
import pandas as pd 
from pandas.errors import EmptyDataError
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import subprocess
from collections import OrderedDict
import warnings
import os, os.path
import glob
import shutil
from Bio.SubsMat.MatrixInfo import blosum62
import matplotlib.pyplot as plt
# from bs4 import BeautifulSoup 
import xml.etree.ElementTree as ET
import urllib

pd.options.mode.chained_assignment = None
#Chrome Driver imports
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException 
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

In [26]:
def create_directory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def remove_thing(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

def empty_directory(path):
    for i in glob.glob(os.path.join(path, '*')):
        remove_thing(i)

def write_errors(gene_name,message,errors_fpath="tmp/errors.tsv"):
    if not os.path.exists(errors_fpath):
        errors_f = open(errors_fpath,'wt')
        errors_f.write("gene\tmessage\n")
        errors_f.close()
    f_line = "{0}\t{1}\n".format(gene_name,message)
    #Check if f_line already in errors_f
    errors_f = open(errors_fpath,'rt')
    errors_flines = errors_f.readlines()
    if f_line not in errors_flines:
        errors_f = open(errors_fpath,'at')
        errors_f.write(f_line)
        errors_f.close()
        
create_directory("tmp")
empty_directory("tmp")

split_parent_dir_path = "reorganized_data"
create_directory(split_parent_dir_path)
split_dir_path = "reorganized_data/hg38"
create_directory(split_dir_path)

In [82]:
FASTA_PATH = "hg38_multiz100way/knownCanonical.exonAA.fa"
TRANSCRIPT_LIST_PATH = "{0}/hg38_transcripts.txt".format(split_parent_dir_path)
transcript_list_f = open(TRANSCRIPT_LIST_PATH,'wt')

curr_ts_id = "@@@"
# header_dict,seq_dict = {}, {}
# block_idx = 0

with open(FASTA_PATH,'rt') as fasta_f: 
    for i in range(60):
        #Iterate over blocks of 201 lines (200 lines corresponding to alternating UCSC Fasta headers and sequence data)
        #Followed by one blank line 
        for j in range(200):
            fline = fasta_f.readline()
            if j == 0:
                #Set ts_id to be the corresponding Ensembl transcript ID for this fasta block. 
                try:
                    ts_id = re.search(">(ENST\d+\.\d+)_",fline).groups()[0]
                except: 
                    #Means extra blank line separating transcripts; read extra line and repeat ts_id extraction
                    #Since end of transcript block of lines, can close out_fasta_f
                    out_fasta_f.close()
                    fline = fasta_f.readline()
                    ts_id = re.search(">(ENST\d+\.\d+)_",fline).groups()[0]                 
                #Start of new transcript block 
                #1: append transcript_id to transcript list file 
                #2: reinitialize all variables for merging each block of UCSC multiz into one line 
                #3: Open new fasta file   
                if not ts_id == curr_ts_id:
                    #Below condition ignores first time initializing new transcript block
                    #Format and write output at this point (single line seq records, eliminate
                    #extraneous information in UCSC headers and keep only transcript/genome name and coordinates)
                    if not curr_ts_id == "@@@":
                        for record_tag in header_dict:
                            header_line = header_dict[record_tag]
                            if record_tag in f_coords_dict:
                                f_coords = f_coords_dict[record_tag]
                                l_coords = l_coords_dict[record_tag]
                                pos_re = "(\d+)\-(\d+)([\-\+])"
                                f_coords_re = re.search(pos_re,f_coords)
                                if record_tag == "ENST00000367772.8_hg38":
                                    print(f_coords_re).groups()
                    curr_ts_id = ts_id
                    transcript_list_f.write(curr_ts_id+"\n")
                    out_fasta_fpath = "{0}/{1}.fasta".format(split_dir_path,curr_ts_id)
                    out_fasta_f = open(out_fasta_fpath,'wt')
                    block_idx = 0
                    header_dict, seq_dict, f_coords_dict, l_coords_dict = {}, {}, {}, {}
            #First block: extract number of blocks from header lines (n_blocks), extract record_tag 
            #(species/ genome) information for each sequence and store 1) first block genome coordinates 
            #(f_coords) 2) header line information (other info encoded in UCSC fasta headers) and 3) first
            #block of sequence data in seq_dict
            if block_idx == 0:
                if j%2 == 0:
                    if j == 0:
                        schema_re = re.search(">ENST\d+\.\d+_[a-zA-Z0-9]+_1_(\d+) ",fline)
                        n_blocks = schema_re.groups()[0]
                    record_tag = re.search("(>ENST\d+\.\d+_[a-zA-Z0-9]+)_",fline).groups()[0]
                    schema_re = re.search(">ENST\d+\.\d+_[a-zA-Z0-9]+_1_\d+ \d+ \d \d ([\w:\+\-]+)",fline)
                    if schema_re: 
                        #If coordinates provided = UCSC record for that species, store in f_coords_dict
                        f_coords = schema_re.groups()[0]
                        f_coords_dict[record_tag] = f_coords
                        l_coords_dict[record_tag] = l_coords
                    header_dict[record_tag] = fline
                elif j%2 == 1:
                    #even lines = sequence data, store in seq_dict
                    seq_dict[record_tag] = fline.strip()
            else:
                if j%2 == 0:
                    record_tag = re.search("(>ENST\d+\.\d+_[a-zA-Z0-9]+)_",fline).groups()[0]
                    if block_idx == int(n_blocks) - 1:
                        schema_re = re.search(">ENST\d+\.\d+_[a-zA-Z0-9]+_\d+_\d+ \d+ \d \d ([\w:\+\-]+)",fline)
                        if schema_re: 
                            l_coords = schema_re.groups()[0]
                            l_coords_dict[record_tag] = l_coords
                    
                elif j%2 == 1:
                    #Append block sequence data into entry in seq_dict
                    prev_line = seq_dict[record_tag]
                    seq_dict[record_tag] = prev_line+fline.strip()
            out_fasta_f.write(fline)
        #End of block. Skip blank line delimiting blocks, increment block_idx
        fasta_f.readline()
        block_idx += 1 
transcript_list_f.close()
out_fasta_f.close()

# print(seq_dict)

chr1:169888676-169888840-
{'>ENST00000367772.8_hg38': 'chr1:169853713-169853772-', '>ENST00000367772.8_panTro4': 'chr1:148251402-148251461-', '>ENST00000367772.8_gorGor3': 'chr1:149059476-149059535-', '>ENST00000367772.8_ponAbe2': 'chr1:81298512-81298571+', '>ENST00000367772.8_nomLeu3': 'chr12:32737555-32737614+', '>ENST00000367772.8_rheMac3': 'chr1:200248663-200248722-', '>ENST00000367772.8_macFas5': 'chr1:27811615-27811674-', '>ENST00000367772.8_papAnu2': 'chr1:192756530-192756589-', '>ENST00000367772.8_chlSab2': 'chr25:59198555-59198614+', '>ENST00000367772.8_calJac3': 'chr18:36609361-36609417+', '>ENST00000367772.8_saiBol1': 'JH378118:36994415-36994471+', '>ENST00000367772.8_otoGar3': 'GL873535:22603272-22603331+', '>ENST00000367772.8_tupChi1': 'KB320708:1027049-1027108+', '>ENST00000367772.8_speTri2': 'JH393291:16828129-16828188+', '>ENST00000367772.8_jacJac1': 'JH725464:27684871-27684930+', '>ENST00000367772.8_micOch1': 'chr6_JH996427_random:13176914-13176967+', '>ENST00000367772

KeyError: '>ENST00000367772.8_apaSpi1'